In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import PIL.Image

In [2]:
path = Path()
codes = np.loadtxt(path/'codes.txt', dtype=str); codes
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

In [3]:
def get_y(o): return path/'labels'/f'{o.stem}_P{o.suffix}'

In [4]:
learn_inf = load_learner(path/'export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [5]:
def on_data_change(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    outputs = learn_inf.predict(img)
    cm_hot = plt.get_cmap('tab10')
    masked = outputs[0].data
    im = np.array(masked)
    im = np.squeeze(im)
    im = cm_hot(im) 
    im = PIL.Image.fromarray((im*255).astype('uint8'))
    background=img.convert("RGBA")
    overlay=im.convert("RGBA")
    background=background.resize((416,416))
    overlay=overlay.resize((416,416))
    new_img = Image.blend(background, overlay, 0.5)
    with out_pl: display(new_img)

In [6]:
btn_upload.observe(on_data_change, names=['data'])

In [7]:
display(VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]))